# Index-Objekte

Die Index-Objekte von pandas sind für die Achsenbeschriftungen und andere Metadaten, wie den Achsennamen, verantwortlich. Jedes Array oder jede andere Sequenz von Beschriftungen, die ihr bei der Konstruktion einer Serie oder eines DataFrame verwendet, wird intern in einen Index umgewandelt:

In [1]:
import pandas as pd

obj = pd.Series(range(7), index=pd.date_range("2022-02-02", periods=7))

In [2]:
obj.index

DatetimeIndex(['2022-02-02', '2022-02-03', '2022-02-04', '2022-02-05',
               '2022-02-06', '2022-02-07', '2022-02-08'],
              dtype='datetime64[ns]', freq='D')

In [3]:
obj.index[3:]

DatetimeIndex(['2022-02-05', '2022-02-06', '2022-02-07', '2022-02-08'], dtype='datetime64[ns]', freq='D')

Indexobjekte sind unveränderlich (immutable) und können daher vom Benutzer nicht geändert werden:

In [4]:
obj.index[1] = '2022-02-03'

TypeError: Index does not support mutable operations

Die Unveränderlichkeit macht die gemeinsame Nutzung von Indexobjekten in Datenstrukturen sicherer:

In [5]:
import numpy as np

labels = pd.Index(np.arange(3))

labels

Int64Index([0, 1, 2], dtype='int64')

In [6]:
obj2 = pd.Series(np.random.randn(3),index=labels)

In [7]:
obj2

0    0.911967
1    1.584217
2    1.987652
dtype: float64

In [8]:
obj2.index is labels

True

Um einem Array ähnlich zu sein verhält sich ein Index auch wie eine Menge mit fester Größe:

In [9]:
data = {'Code': ['U+0000', 'U+0001', 'U+0002', 'U+0003', 'U+0004', 'U+0005'],
        'Decimal': [0, 1, 2, 3, 4, 5],
        'Octal': ['001', '002', '003', '004', '004', '005']}
df = pd.DataFrame(data)

In [10]:
df

,Code,Decimal,Octal
0,U+0000,0,001
1,U+0001,1,002
2,U+0002,2,003
3,U+0003,3,004
4,U+0004,4,004
5,U+0005,5,005


In [11]:
df.columns

Index(['Code', 'Decimal', 'Octal'], dtype='object')

In [12]:
'Code' in df.columns

True

In [13]:
'Key' in df.columns

False

Anders als Python-Sets kann ein Pandas-Index doppelte Label enthalten:

In [14]:
data2 = {'Code': ['U+0006', 'U+0007'],
        'Decimal': [6, 7],
        'Octal': ['006', '007']}
df2 = pd.DataFrame(data2)
dupe = df.append(df2)

dupe

,Code,Decimal,Octal
0,U+0000,0,001
1,U+0001,1,002
2,U+0002,2,003
3,U+0003,3,004
4,U+0004,4,004
5,U+0005,5,005
0,U+0006,6,006
1,U+0007,7,007


Um doppelte Label zu vermeiden, könnt ihr `ignore_index=True` verwenden:

In [15]:
dupe = df.append(df2, ignore_index=True)

dupe

,Code,Decimal,Octal
0,U+0000,0,001
1,U+0001,1,002
2,U+0002,2,003
3,U+0003,3,004
4,U+0004,4,004
5,U+0005,5,005
6,U+0006,6,006
7,U+0007,7,007


Bei Auswahlen mit doppelten Bezeichnungen werden alle Vorkommen der betreffenden Bezeichnung ausgewählt.

Jeder Index verfügt über eine Reihe von Methoden und Eigenschaften für die Mengenlogik, die andere allgemeine Fragen zu den darin enthaltenen Daten beantworten. Im folgenden einige nützliche Methoden und Eigenschaften:

Methode | Beschreibung
:------ | :-----------
`append` | Verketten mit zusätzlichen Indexobjekten, wodurch ein neuer Index entsteht
`difference` | Berechne Differenz zweier Mengen als Index
`intersection` | Berechne Schnittmenge
`union` | Berechne Vereinigungsmenge
`isin` | Berechne boolesches Array, das angibt, ob jeder Wert in der übergebenen Sammlung enthalten ist
`delete` | Berechne neuen Index, wobei das Element in Index `i` gelöscht wird
`drop` | Berechne neuen Index durch Löschen der übergebenen Werte
`insert` | Berechne neuen Index durch Einfügen des Elements in den Index `i`
`is_monotonic` | Gibt `True` zurück, wenn jedes Element größer oder gleich dem vorherigen Element ist
`is_monotonic` | Gibt `True` zurück, wenn der Index keine doppelten Werte enthält
`unique` | Berechnet das Array der eindeutigen Werte im Index

## Neuindizierung

Eine wichtige Methode für Pandas-Objekte ist die Neuindizierung, d.h. die Erstellung eines neuen Objekts mit neu angeordneten Werten, die mit dem neuen Index übereinstimmen. Betrachtet z.B.:

In [16]:
obj = pd.Series(range(7), index=pd.date_range("2022-02-02", periods=7))

In [17]:
obj

2022-02-02    0
2022-02-03    1
2022-02-04    2
2022-02-05    3
2022-02-06    4
2022-02-07    5
2022-02-08    6
Freq: D, dtype: int64

In [18]:
new_index = pd.date_range("2022-02-03", periods=7)

In [19]:
obj.reindex(new_index)

2022-02-03    1.0
2022-02-04    2.0
2022-02-05    3.0
2022-02-06    4.0
2022-02-07    5.0
2022-02-08    6.0
2022-02-09    NaN
Freq: D, dtype: float64

`reindex` erstellt einen neuen Index und indiziert den DataFrame neu. Standardmäßig werden Werte im neuen Index, für die es keine entsprechenden Datensätze im DataFrame gibt, zu `NaN`.

Bei geordneten Daten wie Zeitreihen kann es wünschenswert sein, bei der Neuindizierung Werte zu interpolieren oder zu füllen. Die Option `method` ermöglicht dies mit einer Methode wie `ffill`, die die Werte vorwärts füllt:

In [20]:
obj.reindex(new_index, method='ffill')

2022-02-03    1
2022-02-04    2
2022-02-05    3
2022-02-06    4
2022-02-07    5
2022-02-08    6
2022-02-09    6
Freq: D, dtype: int64

Bei einem DataFrame kann `reindex` entweder den (Zeilen-)Index, die Spalten oder beides ändern. Wenn nur eine Sequenz übergeben wird, werden die Zeilen im Ergebnis neu indiziert:

In [21]:
df.reindex(range(7))

,Code,Decimal,Octal
0,U+0000,0.0,001
1,U+0001,1.0,002
2,U+0002,2.0,003
3,U+0003,3.0,004
4,U+0004,4.0,004
5,U+0005,5.0,005
6,NaN,NaN,NaN


Die Spalten können mit dem Schlüsselwort `columns` neu indiziert werden:

In [22]:
encoding = ['Octal', 'Code', 'Description']

df.reindex(columns=encoding)

,Octal,Code,Description
0,001,U+0000,NaN
1,002,U+0001,NaN
2,003,U+0002,NaN
3,004,U+0003,NaN
4,004,U+0004,NaN
5,005,U+0005,NaN


### Argumente der Funktion `reindex`

Argument | Beschreibung
:------- | :-----------
`labels` | Neue Sequenz, die als Index verwendet werden soll. Kann eine Index-Instanz oder eine andere sequenzähnliche Python-Datenstruktur sein. Ein Index wird genau so verwendet, wie er ist, ohne dass er kopiert wird.
`axis` | Die neu zu indizierende Achse, entweder `index` (Zeilen) oder `columns` (Spalten). Die Vorgabe ist `index`. Ihr könnt alternativ `reindex(index=new_labels)` oder `reindex(columns=new_labels)` verwenden.
`method` | Interpolationsmethode; `ffill` füllt vorwärts, während `bfill` rückwärts füllt.
`fill_value` | Ersatzwert, der zu verwenden ist, wenn fehlende Daten durch Neuindizierung eingefügt werden. Verwendet `fill_value='missing'` (das Standardverhalten), wenn die fehlenden Bezeichnungen im Ergebnis Nullwerte haben sollen.
`limit` |  Beim Vorwärts- oder Rückwärtsfüllen die maximale Anzahl der zu füllenden Elemente.
`tolerance` | Beim Vorwärts- oder Rückwärtsauffüllen die maximale Größe der Lücke, die bei ungenauen Übereinstimmungen gefüllt werden soll.
`level` |  Einfachen Index auf Ebene von `MultiIndex` abgleichen; andernfalls Teilmenge auswählen.
`copy` | Wenn `True`, werden die zugrunde liegenden Daten immer kopiert, auch wenn der neue Index dem alten Index entspricht; wenn `False`, werden die Daten nicht kopiert, wenn die Indizes gleichwertig sind.